<img src="data:image/png;base64, ..." alt="Messier 051" style="float: left; padding: 15px 50px 0px 0px;" />

<i><b>Montage</b> is a general astronomical image toolkit with facilities for reprojection, background matching, mosaicking and visualization.  It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.</i>
    
<i>Montage source code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ).  The Python package can be installed from PyPI ("pip install MontagePy"). See
http://montage.ipac.caltech.edu/ for more information.</i>

# MontagePy.main modules: mImgtbl

A large percentage of the Montage modules focus on processing a set of images (often retrieved from a mission archive); projecting them to a common frame, adjusting their background levels as a set, and coadding them into a mosaic.

Starting with archive access, this mosaicking sequence uses the following modules:

<ul>
    <li><b>mHdr</b> &mdash; Make a template FITS header for a region.</li>
    <li><b>mArchiveList</b> &mdash; Identify archive images for a region.</li>
    <li><b>mArchiveGet</b> and <b>mArchiveExec</b> &mdash; Retreive archive images individually or as a set.</li>
    <li><b>mImgtbl</b> &mdash; Mine a set of images for metadata (and in particular sky coverage).</li>
    <li><b>mCoverageCheck</b> &mdash; Subset an image list based on a region specification.</li>
    <li><b>mMakeHdr</b> or <b>mHdr</b> &mdash; Make a template FITS header bounding a set of images or the header for a sky region.</li>
    <li><b>mProject</b> / <b>mProjectPP</b> / <b>mProjectQL</b> and <b>mProjExec</b> &mdash; Reproject images individually or as a set.</li>
    <li><b>mOverlaps</b> &mdash; Determine which images in a set overlap each other.</li>
    <li><b>mDiff/mFitplane</b> and <b>mDiffFitExec</b> &mdash; Fit the differences between images one pair at a time or all the differences.</li>
    <li><b>mBgModel</b> &mdash; Model a set of differences to determine the "best" background correction for each image.</li>
    <li><b>mBackground</b> and <b>mBgExec</b> &mdash; Background-correct an image or a set of images.</li>
    <li><b>mAdd</b> &mdash; Coadd reprojected (and background corrected) images into a final mosaic.</li>
</ul>

This page is focused on the details of one of these modules: mImgtbl.  For a broader context, please see <a href="Mosaic.ipynb">Building a Mosaic with Montage</a> or the <a href="Mosaic_oneshot.ipynb"> one shot </a> version if you just want to see the commands.

<b>Note:</b> The MontagePy python package has no external dependencies. We include packages like AstroPy on this page to aid in visualizing MontagePy package results.


## mImgtbl Python Help


In [1]:
from MontagePy.main import mImgtbl

help(mImgtbl)


Help on built-in function mImgtbl in module MontagePy.main:

mImgtbl(...)
    mImgtbl extracts the FITS header geometry information from a set of files and creates an ASCII image metadata table which is used by several of the other programs. It only collects data from headers that comply with the FITS standard, but reports a count of images that fail that check.
    
    Parameters
    ----------
    pathname : str
        Directory (or top of tree) for image file search (default to current directory).
    tblname : str
        Output table file name.
    recursiveMode : bool, optional
        Search for images recursively (default just top directory).
    processAreaFiles : bool, optional
        Include 'area' files in the list (not normally useful).
    haveCubes : bool, optional
        False if we know we don't have datacubes (fewer table columns).
    noGZIP : bool, optional
        mImgtbl normally includes analysis of .fits.gz files.  This turns that off.
    showCorners : bool

## mImgtbl Example

mImgtbl scans the input directory (optionally recursively) looking for FITS files with proper world coordinate system (WCS) parameters in the header.  For multi-part files, each HDU is scanned separately.  The results are written to the output table.  These tables are of general utility (<i>e.g.</i>, for constructing searchable archive metadata database tables), but their primary purpose in this mosaicking scenario is as lists of images to process and coverage information to determine image overlaps.

The following call generates a metadata table for a set of 2MASS J-band images near M17 a retreived by mArchiveList/mArchiveExec:

In [2]:
import os

try:
   os.makedirs('work/M17')
except:
   pass

rtn = mImgtbl('M17/raw', 'work/M17/rimages.tbl')

print(rtn)


{'status': '0', 'count': 85, 'badfits': 0, 'badwcs': 0}


Montage modules return JSON structures.  They always include a status (0: success; 1: error) and a variable number of informational parameters.  Here the list contains 85 images.


## Archive List Data

The returned table is in IPAC ASCII format, the default for Montage.  You can read it into a Pandas dataframe with AstroPy.  Here we extract the local file name and the center coordinates.

In [3]:
import os
import numpy as np
import pandas as pd
from astropy.io import ascii

ipactable = ascii.read('work/M17/rimages.tbl').to_pandas()

ipactable

,cntr,ra,dec,cra,cdec,naxis1,naxis2,ctype1,ctype2,crpix1,...,dec1,ra2,dec2,ra3,dec3,ra4,dec4,size,hdu,fname
0,0.0,202.196310,47.652307,13h 28m 47.11s,+47d 39m 08.3s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,47.509894,202.302380,47.794616,202.090260,47.794623,202.090817,47.509901,1488935.0,0.0,2mass-atlas-980527n-j0220256.fits
1,1.0,202.031441,47.326861,13h 28m 07.55s,+47d 19m 36.7s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,47.184471,202.136786,47.469193,201.925983,47.469155,201.926662,47.184432,1493193.0,0.0,2mass-atlas-980527n-j0210033.fits
2,2.0,274.875039,-15.764042,18h 19m 30.01s,-15d 45m 50.5s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,-15.906385,274.949010,-15.621662,274.801044,-15.621674,274.800965,-15.906396,1648112.0,0.0,2mass-atlas-990502s-j1340174.fits
3,3.0,275.752732,-16.516501,18h 23m 00.66s,-16d 30m 59.4s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,-16.658841,275.826978,-16.374119,275.678454,-16.374135,275.678378,-16.658857,1644834.0,0.0,2mass-atlas-990502s-j1470068.fits
4,4.0,275.123042,-15.765737,18h 20m 29.53s,-15d 45m 56.7s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,-15.908077,275.197008,-15.623354,275.049041,-15.623372,275.048973,-15.908094,1599290.0,0.0,2mass-atlas-990502s-j1420174.fits
5,5.0,275.370753,-16.034853,18h 21m 28.98s,-16d 02m 05.5s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,-16.177195,275.444821,-15.892473,275.296658,-15.892486,275.296580,-16.177208,1640108.0,0.0,2mass-atlas-990502s-j1440186.fits
6,6.0,275.246697,-15.709735,18h 20m 59.21s,-15d 42m 35.0s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,-15.852073,275.320638,-15.567350,275.172711,-15.567372,275.172653,-15.852095,1635528.0,0.0,2mass-atlas-990502s-j1430103.fits
7,7.0,274.998829,-15.708226,18h 19m 59.72s,-15d 42m 29.6s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,-15.850565,275.072772,-15.565843,274.924847,-15.565861,274.924782,-15.850584,1651828.0,0.0,2mass-atlas-990502s-j1350103.fits
8,8.0,202.031547,47.057416,13h 28m 07.57s,+47d 03m 26.7s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,46.915026,202.136356,47.199749,201.926626,47.199711,201.927296,46.914988,1492659.0,0.0,2mass-atlas-980527n-j0210044.fits
9,9.0,274.628266,-16.304900,18h 18m 30.78s,-16d 18m 17.6s,512.0,1024.0,RA---SIN,DEC--SIN,256.5,...,-16.447250,274.702453,-16.162527,274.554089,-16.162523,274.553972,-16.447246,1601540.0,0.0,2mass-atlas-001012s-j0140198.fits


## Error Handling

If mImgtbl encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user asks for a non-existent directory:

In [6]:
rtn = mImgtbl('M17/unknown', 'work/M17/rimages.tbl')

print(rtn)


{'status': '1', 'msg': b'Cannot access M17/unknown'}


<p>&nbsp;</p>

## Classic Montage: mImgtbl as a Stand-Alone Program
### mImgtbl Unix/Windows Command-line Arguments

<p>mImgtbl can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mImgtbl [-rcCaidbd][-s statusfile][-f fieldlistfile][-t imglist] directory images.tbl
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mImgtbl can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mImgtbl                                                              */
/*                                                                       */
/*  Montage is a set of general reprojection / coordinate-transform /    */
/*  mosaicking programs.  Any number of input images can be merged into  */
/*  an output FITS file.  The attributes of the input are read from the  */
/*  input files; the attributes of the output are read a combination of  */
/*  the command line and a FITS header template file.                    */
/*                                                                       */
/*  This module, mImgtbl, makes a list (with WCS information) of all     */
/*  FITS image files in the named directory (and optionally recursively) */
/*                                                                       */
/*  mImgtbl supports a lot of variation in input (and a little in        */
/*  output) so there are lot of parameters on the call, most of which    */
/*  can be "defaulted" (i.e. set to zero / NULL).  The full parameter    */
/*  list is:                                                             */
/*                                                                       */
/*   char *pathname        Directory (or top of tree) for image file     */
/*                         search (default to current directory).        */
/*   char *tblname         Output table file name (no default).          */
/*                                                                       */
/*   int recursiveMode     Search for images recursively (default just   */
/*                         top directory).                               */
/*   int processAreaFiles  Include "area" files in the list (not         */
/*                         normally advisable).                          */
/*   int haveCubes         False if we know we don't have datacubes      */
/*                         (fewer table columns)                         */
/*   int noGZIP            mImgtbl normally includes analysis of         */
/*                         .fits.gz files.  This turns that off.         */
/*   int showCorners       Include ra1,dec1, ... dec4 image corner       */
/*                         columns in output.                            */
/*   int showinfo          For HDUs that are are rejected, emit an       */
/*                         INFO message.  Best used in application       */
/*                         mode or when debugging.                       */
/*   int showbad           Show running "INFO" messages for FITS files   */
/*                         that cannot be opened by CFITSIO.             */
/*                                                                       */
/*   char *imgListFile     Rather than searching through directories,    */
/*                         get the list of images from a table file.     */
/*   char *fieldListFile   List of FITS keywords to include in output    */
/*                         table (in addition to the standard WCS info.  */
/*                                                                       */
/*   int debug             Turn on debugging output (not for general     */
/*                         use).                                         */
/*                                                                       */
/*************************************************************************/

struct mImgtblReturn *mImgtbl(char *pathnamein, char *tblname,
                              int recursiveModein, int processAreaFilesin, int haveCubes, int noGZIPin, 
                              int showCornersin, int showinfo, int showbadin, char *imgListFile, char *fieldListFile,
                              int debugin)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mImgtblReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   int    count;         // Number of images found with valid headers (may be more than one per file).
   int    nfile;         // Number of FITS files found.
   int    nhdu;          // Total number of HDSs in all files.
   int    badfits;       // Number of bad FITS files.
   int    badwcs;        // Number of images rejected because of bad WCS information.
};
</pre>